# Preprocessing Notebook

In [ ]:
import pandas as pd
import geopandas as gpd
from ydata_profiling import ProfileReport
import plotly.express as px
import numpy as np


## 1.0 First, air stations pre-processing

In [2]:
air_stations = pd.read_csv('data/air_quality_stations.csv')

In [3]:
air_stations.head(23)

,id,state,latitude,longitude
0,8,ACT,4.530214,-74.142217
1,18,ACT,4.940316,-73.970698
2,20,ACT,5.200000,-73.883000
3,4,ACT,4.596229,-74.194715
4,6,ACT,4.695702,-74.215583
5,17,ACT,4.862871,-74.056341
6,16,ACT,4.298055,-74.819167
7,23,ACT,4.594991,-74.204826
8,14,ACT,4.716525,-74.211712
9,22,ACT,4.809444,-74.102500


In [4]:
# Crear el mapa interactivo con las estaciones
fig = px.scatter_map(
    air_stations,
    lat='latitude',
    lon='longitude',
    hover_name='id',  # Muestra el id al pasar el mouse
    hover_data={
        'latitude': ':.4f',  # Formato con 4 decimales
        'longitude': ':.4f',
        'state': True
    },
    zoom=8,  # Nivel de zoom inicial
    height=600,
    title='Estaciones de Medición de Calidad del Aire - Cundinamarca/Boyacá'
)

# Personalizar el diseño
fig.update_traces(marker=dict(size=12, color='red'))
fig.update_layout(
    map_style="open-street-map",  # Estilo de mapa OpenStreetMap
    margin={"r":0,"t":50,"l":0,"b":0}
)

fig.show()

## 2.0 Emission permits

In [5]:
emission_permits = gpd.read_file('data/emission_permits_anom_2.json')
# Convertir GeoDataFrame a DataFrame normal
emission_permits['latitude'] = emission_permits.geometry.y
emission_permits['longitude'] = emission_permits.geometry.x
emission_permits = pd.DataFrame(emission_permits.drop(columns=['geometry']))

In [6]:
emission_permits.head()

,IDExpediente,Estado,Regional,Departamento,Municipio,Vereda,Class,TipoCombustible,TipoFuenteEmision,Cuenca,latitude,longitude
0,73640.0,Seguimiento y Control,Sabana Occidente,Cundinamarca,MOSQUERA,CENTRO,None,Otros,Horno,Río Bogotá,4.703418,-74.226561
1,73788.0,Seguimiento y Control,Ubate,Cundinamarca,LENGUAZAQUE,Resguardo,None,Carbón,Caldera Horno,Río Suárez,5.318407,-73.704281
2,74314.0,Seguimiento y Control,Sabana Occidente,Cundinamarca,MADRID,LA PUNTA,None,ACPM,Caldera Horno,Río Bogotá,4.800462,-74.210355
3,75972.0,Seguimiento y Control,Sabana Occidente,Cundinamarca,MOSQUERA,Balsillas,None,Fuel Oil No.8,Planta de Asfalto,Río Bogotá,4.678797,-74.284112
4,78824.0,Seguimiento y Control,Sabana Occidente,Cundinamarca,FUNZA,El Hato,None,Carbón,Caldera Horno,Río Bogotá,4.699590,-74.193752


In [7]:
report = ProfileReport(emission_permits, title="Emission Permits Data Profile", explorative=True)
report

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 12/12 [00:00<00:00, 64860.37it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

### Perfilamiento:
- **IDExpediente**: Identificador único, aunque solo 92.3% de los IDs son diferentes, probablemente algunas compañías tienen múltiples ubicaciones
- **Estado**: La mayoría de las estaciones están en estado Sancionatorio (321). Hay 7 "En Trámite" y 216 en "Seguimiento y control".
- **Regional**: De la región donde se ubica el lugar. Puede ser útil para filtros regionales. 14 Distintos.
- **Departamento**: Puede ser útil para filtros departamentales. 3 Distintos
- **Municipio**: Puede ser útil para filtros municipales. 65 Distintos.
- **Vereda**: Muchos valores distintos (148) Omitiría este filtro
- **Class**: 41% de los campos son null. Solo no es null para los permisos de emisión en estado "Sancionatorio".
- **TipoCombustible**: Toma valor "(sin definir)" para permisos en estado "Sancionatorio" y "En Trámite"
- **TipoFuenteEmision**: Igual que el campo anterior
- **Cuenca**: Solo 77 datos no tienen una cuenca asociada
- **latitude** y **longitude**: Hay unos cuantos datos corridos en el mapa. Deben ser arreglados.

In [8]:
emission_permits.drop_duplicates(inplace=True)

### Cuencas, tipos de combustible y de emisión

Los tipos de fuente de emisión muestran valores raros. Debemos ver cómo unificarlos, o dejar ir esta columna.

In [10]:
print("Las cuencas son: ", emission_permits['Cuenca'].unique())
print("================================")
print("Los tipos de combustible son: ", emission_permits['TipoCombustible'].unique())
print("================================")
print("Los tipos de emisión son: ", emission_permits['TipoFuenteEmision'].unique())

Las cuencas son:  ['Río Bogotá' 'Río Suárez' '(sin definir)' 'Río Sumapaz' 'Río Machetá'
 'Río Seco - Río Magdalena' 'Río Negro']
Los tipos de combustible son:  ['Otros' 'Carbón' 'ACPM' 'Fuel Oil No.8' 'Coque' 'NoAplica' 'Madera' 'Gas'
 '(sin definir)' 'Leña' 'Mezcla' 'Hulla']
Los tipos de emisión son:  ['Horno' 'Caldera Horno' 'Planta de Asfalto' 'Caldera' 'Secadores'
 'NoAplica (área de operación)' 'PLANTA DE ASFALTO ADM' 'REACTOR'
 'FILTRO MOLINO PENDULAR' 'Molino' 'Barrilado de grafito ' '(sin definir)'
 'Aspiración molino Danioni I' 'TRITURADOR DE ESCOMBROS' 'HORNO DE SECADO'
 'CAMPANA DE EXTRACCIÓN  PLOMO 1' 'TRITURADORA' 'Chimenea 1'
 'Horno arcillas de Soacha tipo túnel' 'TRITURADOR DE MATERIAL'
 'HORNO TÚNEL 1 SOACHA 2' 'Trituradora' 'Chimenea Triunfo central'
 '700 BHP VR2' 'molino' 'PLANTA DE MEZCLA ASFÁLTICA'
 'Batería de coquización A' 'molino buhler' 'PLANTA TRITURADORA '
 'Batería de producción de coque']


### Reparar la ubicación de algunas de las estaciones

In [ ]:
print("Estadísticas de coordenadas:")
print(f"Latitude - Min: {emission_permits['latitude'].min():.2f}, Max: {emission_permits['latitude'].max():.2f}")
print(f"Longitude - Min: {emission_permits['longitude'].min():.2f}, Max: {emission_permits['longitude'].max():.2f}")

coords_correctas = emission_permits[(emission_permits['longitude'] < -70) & (emission_permits['longitude'] > -75) & 
                                     (emission_permits['latitude'] > 4) & (emission_permits['latitude'] < 7)]
coords_incorrectas = emission_permits[(emission_permits['longitude'] > -70) | (emission_permits['longitude'] < -75) | 
                                       (emission_permits['latitude'] < 4) | (emission_permits['latitude'] > 7)]

print(f"\nPuntos correctos: {len(coords_correctas)}")
print(f"Puntos incorrectos: {len(coords_incorrectas)}")

print("\nMuestra de coordenadas incorrectas:")
print(coords_incorrectas[['IDExpediente', 'Municipio', 'longitude', 'latitude']].head(10))

Estadísticas de coordenadas:
Latitude - Min: 3.89, Max: 86.09
Longitude - Min: -74.82, Max: -39.89

Puntos correctos: 459
Puntos incorrectos: 77

Muestra de coordenadas incorrectas:
    IDExpediente Municipio  longitude   latitude
19      124002.0    SIBATE -40.543104  11.876446
21      127196.0      NILO -74.457576   3.978951
35      136448.0    SOACHA -40.527335  11.886869
36      136490.0  RICAURTE -74.371055   3.887967
38      136886.0    SIBATE -40.541236  11.873642
40      137218.0    SIBATE -40.543524  11.879025
44      139882.0    SOACHA -40.501231  11.869339
60      148522.0    SIBATE -40.541205  11.876106
71      154642.0    SOACHA -40.506221  11.892809
83      166542.0    SIBATE -40.553758  11.868159


In [18]:
puntos_soacha_correctos = coords_correctas[coords_correctas['Municipio'] == 'SOACHA']
puntos_soacha_incorrectos = coords_incorrectas[coords_incorrectas['Municipio'] == 'SOACHA']

print("Puntos SOACHA correctos (muestra):")
print(puntos_soacha_correctos[['IDExpediente', 'longitude', 'latitude']].head(3))
print("\nPuntos SOACHA incorrectos (muestra):")
print(puntos_soacha_incorrectos[['IDExpediente', 'longitude', 'latitude']].head(3))

if len(puntos_soacha_correctos) > 0 and len(puntos_soacha_incorrectos) > 0:
    lon_correcto_promedio = puntos_soacha_correctos['longitude'].mean()
    lat_correcto_promedio = puntos_soacha_correctos['latitude'].mean()
    lon_incorrecto_promedio = puntos_soacha_incorrectos['longitude'].mean()
    lat_incorrecto_promedio = puntos_soacha_incorrectos['latitude'].mean()
    
    delta_lon = lon_correcto_promedio - lon_incorrecto_promedio
    delta_lat = lat_correcto_promedio - lat_incorrecto_promedio
    
    print(f"\nTraslación calculada:")
    print(f"Delta Longitude: {delta_lon:.6f}")
    print(f"Delta Latitude: {delta_lat:.6f}")

Puntos SOACHA correctos (muestra):
    IDExpediente  longitude  latitude
41      137254.0 -74.242432  4.582978
51      142884.0 -74.196453  4.520138
56      145760.0 -74.210626  4.562457

Puntos SOACHA incorrectos (muestra):
    IDExpediente  longitude   latitude
35      136448.0 -40.527335  11.886869
44      139882.0 -40.501231  11.869339
71      154642.0 -40.506221  11.892809

Traslación calculada:
Delta Longitude: -33.703404
Delta Latitude: -7.329896


In [19]:
mask_incorrectos = (emission_permits['longitude'] > -70) | (emission_permits['longitude'] < -75) | \
                   (emission_permits['latitude'] < 4) | (emission_permits['latitude'] > 7)

emission_permits.loc[mask_incorrectos, 'longitude'] = emission_permits.loc[mask_incorrectos, 'longitude'] + delta_lon
emission_permits.loc[mask_incorrectos, 'latitude'] = emission_permits.loc[mask_incorrectos, 'latitude'] + delta_lat

print("Corrección aplicada!")
print(f"\nNuevas estadísticas de coordenadas:")
print(f"Latitude - Min: {emission_permits['latitude'].min():.2f}, Max: {emission_permits['latitude'].max():.2f}")
print(f"Longitude - Min: {emission_permits['longitude'].min():.2f}, Max: {emission_permits['longitude'].max():.2f}")

coords_correctas_final = emission_permits[(emission_permits['longitude'] < -70) & (emission_permits['longitude'] > -75) & 
                                           (emission_permits['latitude'] > 4) & (emission_permits['latitude'] < 7)]
coords_incorrectas_final = emission_permits[(emission_permits['longitude'] > -70) | (emission_permits['longitude'] < -75) | 
                                             (emission_permits['latitude'] < 4) | (emission_permits['latitude'] > 7)]

print(f"\nPuntos correctos después de corrección: {len(coords_correctas_final)}")
print(f"Puntos incorrectos después de corrección: {len(coords_incorrectas_final)}")

Corrección aplicada!

Nuevas estadísticas de coordenadas:
Latitude - Min: -3.44, Max: 78.76
Longitude - Min: -108.16, Max: -73.53

Puntos correctos después de corrección: 533
Puntos incorrectos después de corrección: 3


In [20]:
if len(coords_incorrectas_final) > 0:
    print("Puntos restantes con coordenadas sospechosas:")
    print(coords_incorrectas_final[['IDExpediente', 'Municipio', 'Departamento', 'longitude', 'latitude']])
    
fig_correccion = px.scatter_map(
    emission_permits,
    lat='latitude',
    lon='longitude',
    hover_name='IDExpediente',
    hover_data={
        'Municipio': True,
        'Departamento': True,
        'latitude': ':.4f',
        'longitude': ':.4f'
    },
    zoom=7,
    height=600,
    title='Permisos de Emisión - Coordenadas Corregidas'
)

fig_correccion.update_traces(marker=dict(size=8, color='blue'))
fig_correccion.update_layout(
    map_style="open-street-map",
    margin={"r":0,"t":50,"l":0,"b":0}
)

fig_correccion.show()

Puntos restantes con coordenadas sospechosas:
     IDExpediente Municipio  Departamento   longitude   latitude
21       127196.0      NILO  Cundinamarca -108.160980  -3.350945
36       136490.0  RICAURTE  Cundinamarca -108.074459  -3.441929
228      110204.0     MANTA  Cundinamarca -101.811352  78.757263


### Plantas de emisión - Todos los Estados

In [21]:
color_map = {
    'Sancionatorio': 'red',
    'Seguimiento y Control': 'green',
    'En Trámite': 'black'
}
emission_permits['color'] = emission_permits['Estado'].map(color_map)

fig1 = px.scatter_map(
    emission_permits,
    lat='latitude',
    lon='longitude',
    color='Estado',
    color_discrete_map=color_map,
    hover_name='IDExpediente',
    hover_data={
        'Cuenca': True,
        'Regional': True,
        'Departamento': True,
        'Municipio': True,
        'Estado': False,
        'latitude': False,
        'longitude': False,
        'color': False
    },
    zoom=8,
    height=600,
    title='Permisos de Emisión - Todos los Estados'
)

fig1.update_layout(
    map_style="open-street-map",
    margin={"r":0,"t":50,"l":0,"b":0}
)

fig1.show()

### Plantas de emisión - Estado 'Seguimiento y Control'

In [22]:
seguimiento_permits = emission_permits[emission_permits['Estado'] == 'Seguimiento y Control']
print("Número de permisos en 'Seguimiento y Control': ", len(seguimiento_permits))

fig2 = px.scatter_map(
    seguimiento_permits,
    lat='latitude',
    lon='longitude',
    hover_name='IDExpediente',
    hover_data={
        'Cuenca': True,
        'Regional': True,
        'Departamento': True,
        'Municipio': True,
        'TipoCombustible': True,
        'TipoFuenteEmision': True,
        'latitude': False,
        'longitude': False
    },
    zoom=8,
    height=600,
    title='Permisos de Emisión - Seguimiento y Control'
)

fig2.update_traces(marker=dict(size=12, color='green'))
fig2.update_layout(
    map_style="open-street-map",
    margin={"r":0,"t":50,"l":0,"b":0}
)

fig2.show()

Número de permisos en 'Seguimiento y Control':  210


### Plantas de emisión - Estado Sancionatorio

In [23]:
sancionatorio_permits = emission_permits[emission_permits['Estado'] == 'Sancionatorio']

fig3 = px.scatter_map(
    sancionatorio_permits,
    lat='latitude',
    lon='longitude',
    hover_name='IDExpediente',
    hover_data={
        'Cuenca': True,
        'Regional': True,
        'Departamento': True,
        'Municipio': True,
        'Class': True,
        'latitude': False,
        'longitude': False
    },
    zoom=8,
    height=600,
    title='Permisos de Emisión - Sancionatorio'
)

fig3.update_traces(marker=dict(size=12, color='red'))
fig3.update_layout(
    map_style="open-street-map",
    margin={"r":0,"t":50,"l":0,"b":0}
)

fig3.show()

### Mapa estaciones de medición + emisión
En el mapa a continuación se muestran industrias en estado activo, y estaciones de medición.
- Si la industria está en verde, está en un rango de `RANGO_KM` km de distancia de alguna estación de medición, por lo que podríamos decir que hay una estación midiendo la contaminación de tal industria
- Si la industria está en naranja, no hay estaciones de medición cerca (a menos de `RANGO_KM`km) a ella.

Esto no tiene en cuenta dirección del viento, ni la distancia que viajan diferentes contaminantes, pero es una heurística básica con la que podemos comenzar a trabajar.

In [24]:
from scipy.spatial import distance_matrix

RANGO_KM = 5

def haversine_distance(lat1, lon1, lat2, lon2):
    """Calcula la distancia en km entre dos puntos usando la fórmula de Haversine"""
    R = 6371  # Radio de la Tierra en km
    
    lat1_rad = np.radians(lat1)
    lat2_rad = np.radians(lat2)
    delta_lat = np.radians(lat2 - lat1)
    delta_lon = np.radians(lon2 - lon1)
    
    a = np.sin(delta_lat/2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(delta_lon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    
    return R * c

seguimiento_permits['cercana_a_estacion'] = False
seguimiento_permits['distancia_min_km'] = np.inf
seguimiento_permits['estacion_mas_cercana'] = ''

for idx, permit in seguimiento_permits.iterrows():
    distancias = []
    for _, station in air_stations.iterrows():
        dist = haversine_distance(
            permit['latitude'], permit['longitude'],
            station['latitude'], station['longitude']
        )
        distancias.append((dist, station['id']))
    
    min_dist, closest_station = min(distancias)
    seguimiento_permits.at[idx, 'distancia_min_km'] = round(min_dist, 2)
    seguimiento_permits.at[idx, 'estacion_mas_cercana'] = closest_station
    
    if min_dist <= RANGO_KM:
        seguimiento_permits.at[idx, 'cercana_a_estacion'] = True

cercanas = seguimiento_permits[seguimiento_permits['cercana_a_estacion'] == True]
lejanas = seguimiento_permits[seguimiento_permits['cercana_a_estacion'] == False]

print(f"Rango de análisis: {RANGO_KM} km")
print(f"Total de permisos en Seguimiento y Control: {len(seguimiento_permits)}")
print(f"Permisos cercanos a estaciones (≤{RANGO_KM} km): {len(cercanas)} ({len(cercanas)/len(seguimiento_permits)*100:.1f}%)")
print(f"Permisos lejanos a estaciones (>{RANGO_KM} km): {len(lejanas)} ({len(lejanas)/len(seguimiento_permits)*100:.1f}%)")
print(f"\nDistancia mínima: {seguimiento_permits['distancia_min_km'].min():.2f} km")
print(f"Distancia máxima: {seguimiento_permits['distancia_min_km'].max():.2f} km")
print(f"Distancia promedio: {seguimiento_permits['distancia_min_km'].mean():.2f} km")

Rango de análisis: 5 km
Total de permisos en Seguimiento y Control: 210
Permisos cercanos a estaciones (≤5 km): 134 (63.8%)
Permisos lejanos a estaciones (>5 km): 76 (36.2%)

Distancia mínima: 0.05 km
Distancia máxima: 3800.92 km
Distancia promedio: 42.62 km


/tmp/ipykernel_9381/1098637731.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_9381/1098637731.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_9381/1098637731.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [26]:
seguimiento_validos = seguimiento_permits[seguimiento_permits['distancia_min_km'] < 200].copy()

print(f"\nPermisos válidos (distancia < 200 km): {len(seguimiento_validos)}")
print(f"Permisos descartados por coordenadas erróneas: {len(seguimiento_permits) - len(seguimiento_validos)}")

cercanas_validas = seguimiento_validos[seguimiento_validos['cercana_a_estacion'] == True]
lejanas_validas = seguimiento_validos[seguimiento_validos['cercana_a_estacion'] == False]

print(f"\nEstadísticas corregidas:")
print(f"Permisos cercanos a estaciones (≤{RANGO_KM} km): {len(cercanas_validas)} ({len(cercanas_validas)/len(seguimiento_validos)*100:.1f}%)")
print(f"Permisos lejanos a estaciones (>{RANGO_KM} km): {len(lejanas_validas)} ({len(lejanas_validas)/len(seguimiento_validos)*100:.1f}%)")

air_stations_plot = air_stations.copy()
air_stations_plot['tipo'] = 'Estación de Medición'
air_stations_plot['id_display'] = air_stations_plot['id']

cercanas_plot = cercanas_validas.copy()
cercanas_plot['tipo'] = f'Industria cercana (≤{RANGO_KM} km)'
cercanas_plot['id_display'] = cercanas_plot['IDExpediente'].astype(str)

lejanas_plot = lejanas_validas.copy()
lejanas_plot['tipo'] = f'Industria lejana (>{RANGO_KM} km)'
lejanas_plot['id_display'] = lejanas_plot['IDExpediente'].astype(str)

combined_df = pd.concat([
    air_stations_plot[['latitude', 'longitude', 'tipo', 'id_display']],
    cercanas_plot[['latitude', 'longitude', 'tipo', 'id_display', 'Municipio', 'Regional', 'distancia_min_km', 'estacion_mas_cercana', 'TipoCombustible', 'TipoFuenteEmision']],
    lejanas_plot[['latitude', 'longitude', 'tipo', 'id_display', 'Municipio', 'Regional', 'distancia_min_km', 'estacion_mas_cercana', 'TipoCombustible', 'TipoFuenteEmision']]
], ignore_index=True)

color_map_cobertura = {
    'Estación de Medición': 'blue',
    f'Industria cercana (≤{RANGO_KM} km)': 'green',
    f'Industria lejana (>{RANGO_KM} km)': 'orange'
}

symbol_map = {
    'Estación de Medición': 'star',
    f'Industria cercana (≤{RANGO_KM} km)': 'circle',
    f'Industria lejana (>{RANGO_KM} km)': 'circle'
}

combined_df['symbol'] = combined_df['tipo'].map(symbol_map)

fig_cobertura = px.scatter_map(
    combined_df,
    lat='latitude',
    lon='longitude',
    color='tipo',
    color_discrete_map=color_map_cobertura,
    hover_name='id_display',
    hover_data={
        'Municipio': True,
        'Regional': True,
        'distancia_min_km': ':.2f',
        'estacion_mas_cercana': True,
        'TipoCombustible': True,
        'TipoFuenteEmision': True,
        'latitude': False,
        'longitude': False,
        'tipo': False,
        'symbol': False
    },
    zoom=8,
    height=700,
    title=f'Cobertura de Estaciones de Medición (Rango: {RANGO_KM} km) - Industrias en Seguimiento y Control'
)

for i, trace in enumerate(fig_cobertura.data):
    if 'Estación' in trace.name:
        trace.marker.size = 15
        trace.marker.symbol = 'star'
    else:
        trace.marker.size = 10

fig_cobertura.update_layout(
    map_style="open-street-map",
    margin={"r":0,"t":50,"l":0,"b":0},
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01,
        bgcolor="rgba(255, 255, 255, 0.8)"
    )
)

fig_cobertura.show()


Permisos válidos (distancia < 200 km): 208
Permisos descartados por coordenadas erróneas: 2

Estadísticas corregidas:
Permisos cercanos a estaciones (≤5 km): 134 (64.4%)
Permisos lejanos a estaciones (>5 km): 74 (35.6%)
